# The Twitter data transformation project

The main aim of this project is to extract tweets and related data from twitter users using the twitter API, the twitter data may include information about the tweets, like, comments, retweets, among others. 
The data scraped will be stored in a csv file which makes it available for other form of cleaning, transformation and data wrangling.
The needed libraries will be imported below to make sure everything is up and running. Then we get access to use the twitter API from the jupyter notebook.

In [43]:
# Here we are installing the librabries needed in the few lines of code below, 
# The tweepy library helps us get access and scrape data from the twitter user or their timeline.
# The pandas is used for most of the data exploration, and wrangling.
# The numpy will be used for data analysis if needs be...
# The spacy is a open-source library for  Natural Language Processing 
# Re is Regular Expression opreations
# Webbrowser is a library used to access a browser in the notebook environment.
# Time library is also imported.

import tweepy 
import pandas as pd
import numpy as np 
import webbrowser 
import spacy
import time
import re

In [44]:
#Here I run a different jupyter notebook where I saved my tweepy credentials.
%run ./Keys.ipynb

For the next cell, I did the tweepy authentication. Which later gave me the access to use  the API to scrape data from the twitter application itself.
The consumer_key, consumer_secret variables are saved in the keys.ipynb notebook which I ran in cell 2

In [45]:
callback_uri = "oob"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret, callback_uri)
redirect_url = auth.get_authorization_url()
webbrowser.open(redirect_url)
user_pin_input = input("What is the use pin value? ")

What is the use pin value? 3724207


You can run this line of code, then cross check the access_token with those you saved in your credentials/keys notebook.
**auth.get_access_token(user_pin_input)**

In [46]:
# This cell gets you access to use the API within your access limitations.
api = tweepy.API(auth)

In [133]:
#auth.get_access_token(user_pin_input)

In [52]:
#To check for that your in the correct user or your personal twitter timeline.
me = api.me()
print(me.screen_name)

FemdamCladof


In [53]:
def extract_twitter_timeline(timeline):
    
    """
    This function is created to take tweets and few of its attributes e.g the tweets, the likes, the retweets e.t.c
    It takes only two types of data types(i.e strings and integers) form the timeline.
    
    Parameter:
    It takes only one parameter which is the timeline of the account to be scraped.
      e.g  timeline = api.get_user("user")
    
    The function returns a DataFrame of the data from twitter with the author of the tweets appended,
    alos the owner of the twitter account.
    
    """
    
    columns = set()
    allowed_types = [str, int]
    tweet_data = []
    
    for item in timeline:
        item_dict = dict(vars(item))
        item_keys = item_dict.keys()
        single_tweets_data = {"user": item.user.screen_name, "author": item.author.screen_name}
        
        for k in item_keys:
            try:
                v_type = type(item_dict[k])
            except: v_type = None
            if v_type != None:
                    if v_type in allowed_types:
                        single_tweets_data[k] = item_dict[k]
                    columns.add(k)
                    
        tweet_data.append(single_tweets_data)
        
        
        
    header_cols = list(columns)
    header_cols.append("user")
    header_cols.append("author")
    df = pd.DataFrame(tweet_data, columns = header_cols)
    return df
       

In [54]:
# I am going to use the above function to extract data from my timeline ...
femdam = me.timeline()
femdam_timeline = extract_twitter_timeline(femdam)

In [55]:
# Here is what it looks like...
femdam_timeline.head()

,retweeted,coordinates,truncated,favorite_count,entities,favorited,lang,source_url,source,possibly_sensitive,...,geo,text,id,in_reply_to_screen_name,in_reply_to_status_id_str,in_reply_to_user_id_str,retweet_count,contributors,user,author
0,NaN,NaN,NaN,0,NaN,NaN,en,http://twitter.com/download/android,Twitter for Android,NaN,...,NaN,RT @JonErlichman: On this day in 1976: Apple r...,1381360568796385282,NaN,NaN,NaN,1202,NaN,FemdamCladof,FemdamCladof
1,NaN,NaN,NaN,0,NaN,NaN,en,http://twitter.com/download/android,Twitter for Android,NaN,...,NaN,RT @towernter: Machine Learning engineers trai...,1376881640249626624,NaN,NaN,NaN,682,NaN,FemdamCladof,FemdamCladof
2,NaN,NaN,NaN,0,NaN,NaN,en,http://twitter.com/download/android,Twitter for Android,NaN,...,NaN,RT @svpino: My recommendation to learn machine...,1371608603988258816,NaN,NaN,NaN,473,NaN,FemdamCladof,FemdamCladof
3,NaN,NaN,NaN,0,NaN,NaN,en,http://twitter.com/download/android,Twitter for Android,NaN,...,NaN,RT @josh_tobin_: Why do people always ask what...,1370264117198618627,NaN,NaN,NaN,284,NaN,FemdamCladof,FemdamCladof
4,NaN,NaN,NaN,1,NaN,NaN,en,http://twitter.com/download/android,Twitter for Android,NaN,...,NaN,"@jeafreezy The question is: ""Do we have a data...",1352369721602174980,jeafreezy,1352367402357551112,2753232592,0,NaN,FemdamCladof,FemdamCladof


In [56]:
# Getting the text attribute from my timeline.
femdam_timeline.text

0     RT @JonErlichman: On this day in 1976: Apple r...
1     RT @towernter: Machine Learning engineers trai...
2     RT @svpino: My recommendation to learn machine...
3     RT @josh_tobin_: Why do people always ask what...
4     @jeafreezy The question is: "Do we have a data...
5                @SirFabulous5 You have time traveled??
6                This is great❤ https://t.co/ABOKb7Ggwn
7     RT @getpneumacare: Don't self medicate! Talk t...
8                             ❤ https://t.co/V80SjowMAl
9     How awesome could this be..?\nYou have no excu...
10    RT @iyoaye: Full-stack developer position at @...
11    RT @getpneumacare: We started with a mission t...
12    We created magic at https://t.co/Q1R0wcWj93 an...
13    If you're a interested in a remote mapathon an...
14       This guy is the best😍😘 https://t.co/c6nHUiFEM4
Name: text, dtype: object

## SCRAPING DATA TWEETS FROM TWITTER AND STORING THEM IN A CSV FOLDER###

This is becoming more fun as we go on...
We have been able to extract twitter data from my personal timeline, Now we try to get data from other users timeline.
We will also use the extract_twitter_timeline function in the aspect of the work.

### scraping data from Adeyemo Ayodele. O (@iyoaye)

In [57]:
# defining a function to scarp tweets data from a particular user.

def extract_twitter_timeline(timeline):
    
    """
    This model is created to take tweets and all associated data e.g the tweets, the likes, the retweets e.t.c
    It takes the only two types of data types from the timeline, It takes both the string and integer data types.
    It takes the timeline of the account to be scraped and returns a DataFrame of the data from twitter.
    
    """
    
    columns = set()
    allowed_types = [str, int]
    tweet_data = []
    
    for item in timeline:
        item_dict = dict(vars(item))
        item_keys = item_dict.keys()
        single_tweets_data = {"user": item.user.screen_name, "author": item.author.screen_name}
        
        for k in item_keys:
            try:
                v_type = type(item_dict[k])
            except: v_type = None
            if v_type != None:
                    if v_type in allowed_types:
                        single_tweets_data[k] = item_dict[k]
                    columns.add(k)
                    
        tweet_data.append(single_tweets_data)
        
        
        
    header_cols = list(columns)
    header_cols.append("user")
    header_cols.append("author")
    df = pd.DataFrame(tweet_data, columns = header_cols)
    return df
       

There are few twitter users which I personally will like to explore their timeline, I will start with Adeyemo Ayodele O.
with twitter handle "iyoaye"... Let's get it!

In [58]:
# We need to store the user in a variable then call the timeline function on the variable,
#to get the timeline data of the individual
ayodele = api.get_user("iyoaye")
ay_timeline = ayodele.timeline()

In [59]:
# There are a lot of attributes associated with the data from each timeline,
#thats what we keep in the attributes variable below:
attributes = dir(ay_timeline)

Now let's use the extract_twitter_timeline function to get the data into a pandas dataframe so it makes more sense.

In [60]:
iyoaye = extract_twitter_timeline(ay_timeline)
iyoaye.head()

,retweeted,coordinates,truncated,favorite_count,entities,favorited,lang,source_url,source,possibly_sensitive,...,geo,text,id,in_reply_to_screen_name,in_reply_to_status_id_str,in_reply_to_user_id_str,retweet_count,contributors,user,author
0,NaN,NaN,NaN,0,NaN,NaN,und,http://twitter.com/download/android,Twitter for Android,NaN,...,NaN,💯 https://t.co/5eJDpZzBxj,1396453081630625792,NaN,NaN,NaN,0,NaN,iyoaye,iyoaye
1,NaN,NaN,NaN,0,NaN,NaN,en,http://twitter.com/download/android,Twitter for Android,NaN,...,NaN,@O_Gbemiro Thanks man for the reference.\n\nWe...,1396452986096861187,O_Gbemiro,1396439056804990981,73132987,0,NaN,iyoaye,iyoaye
2,NaN,NaN,NaN,0,NaN,NaN,en,http://twitter.com/download/android,Twitter for Android,NaN,...,NaN,RT @httpsgallery: Sometimes the best therapy i...,1396035675808489478,NaN,NaN,NaN,38037,NaN,iyoaye,iyoaye
3,NaN,NaN,NaN,0,NaN,NaN,en,http://twitter.com/download/android,Twitter for Android,NaN,...,NaN,RT @docolumide: Amazon is no longer the top sh...,1395580022958837760,NaN,NaN,NaN,3,NaN,iyoaye,iyoaye
4,NaN,NaN,NaN,1,NaN,NaN,tl,http://twitter.com/download/android,Twitter for Android,NaN,...,NaN,@wandesak Hahah,1395530625986224130,wandesak,1395527696566861828,172325571,0,NaN,iyoaye,iyoaye


In [61]:
#This line of code returns the text attribute from the iyoaye variable which we extracted the twitter data as a dataframe into.
#It includes strings, links and a lot of unwanted characters which will be dealt with later.
iyoaye.text

0                             💯 https://t.co/5eJDpZzBxj
1     @O_Gbemiro Thanks man for the reference.\n\nWe...
2     RT @httpsgallery: Sometimes the best therapy i...
3     RT @docolumide: Amazon is no longer the top sh...
4                                       @wandesak Hahah
5                             @Sisi_Ose I like it too..
6     RT @thisisAQ: Having the most powerful voice i...
7     RT @ayobamiteriba: Health Care companies are n...
8     Bros, it was a great pleasure connecting. http...
9                  There is no home from home anywhere.
10    If you'd like to build a @ModernFertility for ...
11                         @KiraMcCroden Interesting...
12    Who is building a @ModernFertility for the Nig...
13                      What!!! https://t.co/NGtYYMpVDG
14    Thanks to everyone that has responded and shar...
15                                      @Sisi_Ose 🚶‍♂️✌
16                                          @Sisi_Ose 👀
17    # let the 'dip' call unto other 'dips'\nWh

In the cell below, another function is created and it generate_twitter_data from user, It takes a few of the attributes.
The function will be used to get the data from the remaining users.

In [62]:
def generate_twitter_data(ID, number_of_tweets):
    
    """
    This function(generate_twitter_data) takes in the twitter handle of the individual as the ID parameter and the number
    of expected tweets as the number_of_tweets parameter. 
    The data (e.g tweets, time of tweet, number of likes, the tweeter e.t.c) is collected in to lists,
    which later gets converted to a pandas DataFrame.
    The function returns the dataframe.
        
    parameters:
    ID = "twitter handle" as a string type object.
    number_of_tweets = number of expected tweets as an interger.   
    
    """

    likes = []
    time = []
    tweets = []
    tweeter = []

    
    for i in tweepy.Cursor(api.user_timeline, ID, tweet_mode = "extended").items(number_of_tweets):
        likes.append(i.favorite_count)
        time.append(i.created_at)
        tweets.append(i.full_text)
        tweeter.append(i.user.screen_name)
    df = pd.DataFrame({"Time": time, "Tweets": tweets, "Tweeter": tweeter, "Likes": likes})
    return df
   

### Generating twitter data from OurFavOnlineDoc 

Another user I want to get tweets from is the "OurFavOnlineDoc" with the hanlde "DrOlufunmilayo"

In [64]:
fav =  generate_twitter_data("DrOlufunmilayo", 200)

In [65]:
# Just taking a look at a sample of the twitter data generated in a dataframe.
fav.head(10)

,Time,Tweets,Tweeter,Likes
0,2021-05-25 12:10:32,RT @ChidiOdinkalu: After the opening para of @...,DrOlufunmilayo,0
1,2021-05-25 10:03:44,Sack the motivational speaker handling your pa...,DrOlufunmilayo,690
2,2021-05-25 09:54:47,RT @spiciousOmar: Please tell your yard people...,DrOlufunmilayo,0
3,2021-05-25 09:47:17,RT @TheSilverClock1: Pls let join and hear fro...,DrOlufunmilayo,0
4,2021-05-25 09:45:16,Happy World Africa Day!\nI will be discussing ...,DrOlufunmilayo,111
5,2021-05-25 08:29:05,RT @capitol_51926: @DrOlufunmilayo Bro I opene...,DrOlufunmilayo,0
6,2021-05-25 08:28:41,Happy birthday 🥳 https://t.co/1o8j0B0RE4,DrOlufunmilayo,45
7,2021-05-25 08:28:31,RT @Risingphoennix: @DrOlufunmilayo Thanks doc...,DrOlufunmilayo,0
8,2021-05-25 07:37:40,Today is Day 2 #NoFizzyDrinksWeek \nWe will st...,DrOlufunmilayo,1167
9,2021-05-24 21:32:46,Unknown gun babes don enter my page https://t....,DrOlufunmilayo,207


### Generating twitter data from Aproko Doctor with the handle "aproko_doctor"

In [67]:
# We are also using the generate_twitter_data function here.
aproko = generate_twitter_data("aproko_doctor", 200)
aproko.head()

,Time,Tweets,Tweeter,Likes
0,2021-05-25 11:15:53,"RT @aproko_doctor: Okiriki is back, Ikegwuru w...",aproko_doctor,0
1,2021-05-25 10:49:12,This new trend of flashing lights that many Ok...,aproko_doctor,2706
2,2021-05-25 10:43:58,"@adesoji45 @chef_amakaa Chief, we need an inqu...",aproko_doctor,4
3,2021-05-25 10:34:18,@chef_amakaa So you cannot tell them who shot ...,aproko_doctor,136
4,2021-05-25 10:32:47,RT @chef_amakaa: For your eyes only😁 https://t...,aproko_doctor,0


In [68]:
# Trying to get the data same way, let's get that of iyoaye using the same method.
ay_tweets = generate_twitter_data("iyoaye", 200)
ay_tweets

,Time,Tweets,Tweeter,Likes
0,2021-05-23 13:08:54,💯 https://t.co/5eJDpZzBxj,iyoaye,0
1,2021-05-23 13:08:31,@O_Gbemiro Thanks man for the reference.\n\nWe...,iyoaye,0
2,2021-05-22 09:30:16,RT @httpsgallery: Sometimes the best therapy i...,iyoaye,0
3,2021-05-21 03:19:40,RT @docolumide: Amazon is no longer the top sh...,iyoaye,0
4,2021-05-21 00:03:23,@wandesak Hahah,iyoaye,1
...,...,...,...,...
195,2021-04-25 21:46:03,Hi @chinasza https://t.co/27huqWT2aI,iyoaye,0
196,2021-04-25 21:36:52,Revenue! Revenue!! Revenue!!! Everything else ...,iyoaye,14
197,2021-04-24 23:33:06,RT @FeelGoodPage11: Today I saw the impossible...,iyoaye,0
198,2021-04-24 22:34:20,Might also be worth checking out @CarequalityN...,iyoaye,1


For the purpose of this project we need to do some data cleaning, which is very important for every project.
We have to remove the following rows:
    1. Rows with links.
    2. Rows that are retweeted.
    3. Rows with the @ sign.
    
To perform the following tasks we have to create a function, The function is created in the cell below.

In [94]:
def clean_tweets(df):
    df = df[~df.Tweets.str.contains("RT")]
    df = df[~df.Tweets.str.contains("https")]
    df = df[~df.Tweets.str.contains("@")]
    df.reset_index(drop=True)
    return df

In [70]:
clean_tweets(fav)

,Time,Tweets,Tweeter,Likes
21,2021-05-24 20:05:49,You are complaining about Lagos restaurants bu...,DrOlufunmilayo,3056
32,2021-05-24 13:20:00,Please let’s help save Professor Adenuga. If y...,DrOlufunmilayo,140
34,2021-05-24 13:18:03,"Unlike many others,\nTo everyone who knew him,...",DrOlufunmilayo,156
47,2021-05-23 21:25:54,Your friend’s wedding is almost here.\nContinu...,DrOlufunmilayo,1337
52,2021-05-23 20:59:49,"Always better to read bible,\nThan listen to p...",DrOlufunmilayo,1282
54,2021-05-23 12:51:31,“Submissive” in many cases only means:\nEasy t...,DrOlufunmilayo,643
55,2021-05-23 09:22:23,Sorry. The Senate President was there.\nBut sa...,DrOlufunmilayo,283
58,2021-05-23 08:48:13,The President.\nVice President.\nSenate Presid...,DrOlufunmilayo,1828
60,2021-05-23 07:34:14,Buhari was a former General.\nYet as the dead ...,DrOlufunmilayo,1368
61,2021-05-22 23:55:03,Imagine you work in a hospital as a doctor/nur...,DrOlufunmilayo,1040


We have generated data from 4 twitter users and we got 200 rows from each user. 
Now we have join all the four dataframes and with the help Pandas concantenate function we can join it easily.
The cell below does the concatenating and stores them in another new DataFrame "twitter_df".

In [85]:
twitter_df = pd.concat([aproko, fav, buhari, ay_tweets])
twitter_df

,Time,Tweets,Tweeter,Likes
0,2021-05-25 11:15:53,"RT @aproko_doctor: Okiriki is back, Ikegwuru w...",aproko_doctor,0
1,2021-05-25 10:49:12,This new trend of flashing lights that many Ok...,aproko_doctor,2706
2,2021-05-25 10:43:58,"@adesoji45 @chef_amakaa Chief, we need an inqu...",aproko_doctor,4
3,2021-05-25 10:34:18,@chef_amakaa So you cannot tell them who shot ...,aproko_doctor,136
4,2021-05-25 10:32:47,RT @chef_amakaa: For your eyes only😁 https://t...,aproko_doctor,0
...,...,...,...,...
195,2021-04-25 21:46:03,Hi @chinasza https://t.co/27huqWT2aI,iyoaye,0
196,2021-04-25 21:36:52,Revenue! Revenue!! Revenue!!! Everything else ...,iyoaye,14
197,2021-04-24 23:33:06,RT @FeelGoodPage11: Today I saw the impossible...,iyoaye,0
198,2021-04-24 22:34:20,Might also be worth checking out @CarequalityN...,iyoaye,1


We simply converted the Time column of the DataFrame to a datetime data-type, so that we can perform some time functions on it.

In [93]:
twitter_df.Time = pd.to_datetime(twitter_df.Time)
twitter_df.head()

,Time,Tweets,Tweeter,Likes
0,2021-05-25 12:10:32,RT @ChidiOdinkalu: After the opening para of @...,DrOlufunmilayo,0
0,2021-05-25 11:15:53,"RT @aproko_doctor: Okiriki is back, Ikegwuru w...",aproko_doctor,0
1,2021-05-25 10:49:12,This new trend of flashing lights that many Ok...,aproko_doctor,2706
2,2021-05-25 10:43:58,"@adesoji45 @chef_amakaa Chief, we need an inqu...",aproko_doctor,4
3,2021-05-25 10:34:18,@chef_amakaa So you cannot tell them who shot ...,aproko_doctor,136


The Dataframe is sorted with  by the Time column and it is saved.

In [88]:
twitter_df.sort_values(by=["Time"], ascending=False, inplace=True)

We call the clean_tweets function on our dataframe to clean them of the unwanted characters in the Tweet column.

In [95]:
cleaned_twitter_df = clean_tweets(twitter_df)

Since we removed some of the rows then it is certain that our index will be altered and we need to reset it.

We reset our index with the following line of code.

In [98]:
cleaned_twitter_df.reset_index(drop=True, inplace = True)

I have a csv file saved already where I have twitter data I already scraped...

I will import it to the notebook, using the next cell of code.

In [99]:
tweetdata = pd.read_csv("./TweetData.csv", index_col=0, encoding="UTF-8-sig")
tweetdata = tweetdata.reset_index(drop= True)

In [100]:
tweetdata.head()

,Time,Tweets,Tweeter,Likes
0,2021-05-18 10:02:12,A child has been potentially permanently scarr...,DrOlufunmilayo,172
1,2021-05-18 09:12:33,"Since you all are mad,\nPls remember who the a...",DrOlufunmilayo,705
2,2021-05-18 08:01:20,"If you're not careful, social media will have ...",aproko_doctor,10948
3,2021-05-18 07:41:26,A child got poured kerosene.\nA child got set ...,DrOlufunmilayo,629
4,2021-05-18 07:06:14,"“As a journalist, it’s unprofessional of him t...",DrOlufunmilayo,769


I have read the saved csv data in to the tweetdata variable and I will concatnate it with the twitter_df.

Then change the datatype of the Time column.

Also sort the values by the Time column, hence reset index.

In [104]:
twitter_data = pd.concat([twitter_df, tweetdata])

In [108]:
twitter_data.Time = pd.to_datetime(twitter_data["Time"])

In [109]:
twitter_data.sort_values(by=["Time"], ascending=False, inplace =True)

In [111]:
twitter_data.reset_index(drop= True)

,Time,Tweets,Tweeter,Likes
0,2021-05-25 12:10:32,RT @ChidiOdinkalu: After the opening para of @...,DrOlufunmilayo,0
1,2021-05-25 11:15:53,"RT @aproko_doctor: Okiriki is back, Ikegwuru w...",aproko_doctor,0
2,2021-05-25 10:49:12,This new trend of flashing lights that many Ok...,aproko_doctor,2706
3,2021-05-25 10:43:58,"@adesoji45 @chef_amakaa Chief, we need an inqu...",aproko_doctor,4
4,2021-05-25 10:34:18,@chef_amakaa So you cannot tell them who shot ...,aproko_doctor,136
...,...,...,...,...
1069,2020-11-30 17:31:51,We will intensify our cooperation with neighbo...,MBuhari,1302
1070,2020-11-30 17:30:34,"As we mourn all the lives lost in Zabarmari, t...",MBuhari,1072
1071,2020-11-30 17:28:23,Nothing is more important than ensuring the se...,MBuhari,9605
1072,2020-11-25 20:00:02,I wish you successful commemoration and Days o...,MBuhari,647


After concatnating both dataframes, it is important that we check for duplicates. 

We must drop duplicates and drop one of them, which makes our data good and tidy enough.

In [115]:
twitter_data.drop_duplicates(inplace =True)
twitter_data.shape

(456, 4)

There are differnce in the encoding version of the original twitter objects and the one we are using in this notebook.

I have tried to use a particular encoding which helps to convert emojis and the quotation marks.

But I was unable to find one that will change the whitespaces from "\n\n" to " " or "\n" to " ".

So I decided to create a function that takes the dataframe as a parameter.

The function is created below.

In [101]:
def remove_whitespaces(tweetdata):
    tweetdata["Tweets"] = tweetdata['Tweets'].apply(lambda x: x.replace('\n', ' '))
    tweetdata["Tweets"] = tweetdata['Tweets'].apply(lambda x: x.replace('\n', ' '))
    return tweetdata

calling the function on our dataframe... This helped to clean the unwanted characters with whitespaces

In [116]:
remove_whitespaces(tweetdata)

,Time,Tweets,Tweeter,Likes
0,2021-05-18 10:02:12,A child has been potentially permanently scarr...,DrOlufunmilayo,172
1,2021-05-18 09:12:33,"Since you all are mad, Pls remember who the ac...",DrOlufunmilayo,705
2,2021-05-18 08:01:20,"If you're not careful, social media will have ...",aproko_doctor,10948
3,2021-05-18 07:41:26,A child got poured kerosene. A child got set a...,DrOlufunmilayo,629
4,2021-05-18 07:06:14,"“As a journalist, it’s unprofessional of him t...",DrOlufunmilayo,769
...,...,...,...,...
269,2020-11-30 17:31:51,We will intensify our cooperation with neighbo...,MBuhari,1302
270,2020-11-30 17:30:34,"As we mourn all the lives lost in Zabarmari, t...",MBuhari,1072
271,2020-11-30 17:28:23,Nothing is more important than ensuring the se...,MBuhari,9605
272,2020-11-25 20:00:02,I wish you successful commemoration and Days o...,MBuhari,647


In [121]:
twitter_data.sort_values(by=["Time"], ascending = False, inplace = True)

In [124]:
twitter_data.reset_index(drop = True, inplace = True)

In [129]:
twitter_data = remove_whitespaces(twitter_data)
twitter_data

,Time,Tweets,Tweeter,Likes
0,2021-05-25 10:49:12,This new trend of flashing lights that many Ok...,aproko_doctor,2706
1,2021-05-25 07:22:10,"Let's make this clear, a person can have a sei...",aproko_doctor,413
2,2021-05-25 07:22:10,"Epilepsy can develop at any age, But is usual...",aproko_doctor,99
3,2021-05-25 07:22:09,For a lot of people we can't tell what led to ...,aproko_doctor,69
4,2021-05-25 07:22:09,a lack of oxygen at birth diseases such as AID...,aproko_doctor,73
...,...,...,...,...
451,2020-11-30 17:31:51,We will intensify our cooperation with neighbo...,MBuhari,1302
452,2020-11-30 17:30:34,"As we mourn all the lives lost in Zabarmari, t...",MBuhari,1072
453,2020-11-30 17:28:23,Nothing is more important than ensuring the se...,MBuhari,9605
454,2020-11-25 20:00:02,I wish you successful commemoration and Days o...,MBuhari,647


In [132]:
df = pd.read_csv("./twitter_data.csv", index_col=0)
df.head()

,Time,Tweets,Tweeter,Likes
0,2021-05-25 10:49:12,This new trend of flashing lights that many Ok...,aproko_doctor,2706
1,2021-05-25 07:22:10,"Let's make this clear, a person can have a sei...",aproko_doctor,413
2,2021-05-25 07:22:10,"Epilepsy can develop at any age, But is usual...",aproko_doctor,99
3,2021-05-25 07:22:09,For a lot of people we can't tell what led to ...,aproko_doctor,69
4,2021-05-25 07:22:09,a lack of oxygen at birth diseases such as AID...,aproko_doctor,73


To save the dataframe into a csv file, The pandas library has a function "to_csv" which is used in the cell below.

In [130]:
twitter_data.to_csv("twitter_data.csv")

**Conclusions**


In this course of this of this project, I was able to access my twitter timeline from the jupyter notebook.
Then we were able to extract the data from the twitter users, also I was able to clean them off some unwanted characters among other data wrangling progresses.
Finally, we were able to save the dataframe into a csv file.


Thanks!